In [77]:
import requests
import numpy as np
import pandas as pd
%run ../employee/links.ipynb

In [78]:
# req = requests.get(links_ns.taskrole.task+"4").json() # Risk of empty list
req = requests.get(links_ns.task.project+"4").json() # Risk of empty list
task = pd.json_normalize(req)


In [79]:
task

""


### 1.Task bio data

In [80]:
# Request list of project tasks
req = requests.get(links_ns.task.single+"4").json() # Risk of empty list or 404
task = pd.json_normalize(req)
task.to_dict(orient="records")

[{'taskId': 4,
  'code': '350511A23CB0107',
  'name': 'État des lieux du secteur des transports dans les différents districts ',
  'type': 'PROJECT',
  'startDate': '2023-01-02',
  'endDate': '2023-06-30',
  'status': 'SCHEDULED',
  'progress': None,
  'budget': None,
  'comment': 'DCSTI',
  'project': None}]

### 2.Project Stats

In [81]:
data = dict()
data_task = dict()
data_resources = dict()

id = 42

req = requests.get(links_ns.task.project+str(id))
if req.status_code == 200 and len(req.json())>0:
    task = pd.json_normalize(req.json())
    data_task = task
    nb_task = task.shape[0] # First value
    # For each task compute number of employees involved
    nb_emp = list([])
    role_tmp = pd.DataFrame()
    for taskId in task.taskId:
        req = requests.get(links_ns.taskrole.task+str(taskId)).json() # Risk of empty list
        if len(req)>0:
            role = pd.json_normalize(req)
            role_tmp = pd.concat([role_tmp, role])
            nb_emp.append(role["position.positionId"].values)
    data_resources = role_tmp
    if len(nb_emp)>0:
        nb_emp = len(set(np.concatenate(nb_emp))) # Second value
    else: nb_emp=0
    # Project budget
    req = requests.get(links_ns.task.single+str(id)).json() # Risk of empty list
    task = pd.json_normalize(req)
    budget = task.budget.max()
    data = {"tasks":nb_task,"resources":nb_emp,"budget":budget}
    # print(nb_task, nb_emp, budget)
else: # retuen empty dict
    data = {"tasks":0,"resources":0,"budget":0}
data

{'tasks': 11, 'resources': 0, 'budget': nan}

### 3. Activity List

In [82]:
pd.DataFrame([data])

,tasks,resources,budget
0,11,0,NaN


In [83]:

resources_col = dict({
    "role":None,
    "task.taskId":None,
    "position.employee.employeeId":None,
    "position.employee.firstName":None,
    "position.employee.lastName":None
})

df = pd.DataFrame()
tasks_df = data_task[["taskId","code","name","type","status","endDate"]].copy()

if data_resources.shape[0]==0:
    resources_df = pd.DataFrame([resources_col])
    df = pd.concat([tasks_df, resources_df])

#df = pd.merge(left=tasks_df, right=resources_df, left_on="taskId", right_on="task.taskId")
df.drop(columns=["task.taskId"], inplace=True)
rename_col = ["taskId","code","name","type","status","endDate","role","employeeId","firstName","lastName"]
df.columns = rename_col
df.reset_index(inplace=True, drop=["index"])
df = df.drop(df.index[-1])
df

,taskId,code,name,type,status,endDate,role,employeeId,firstName,lastName
0,79.0,110306A23CB0067,Achat de 7 véhicules de fonction,ACTIVITY,SCHEDULED,2023-12-29,NaN,NaN,NaN,NaN
1,80.0,110303A23CB0064,Construction batiemets annexes siège,ACTIVITY,SCHEDULED,2023-03-31,NaN,NaN,NaN,NaN
2,81.0,110334A23CB0095,Formations DMGP,ACTIVITY,SCHEDULED,2023-12-29,NaN,NaN,NaN,NaN
3,82.0,110307A23CB0068,Projet d'aquisition du local bureau Bouaké,ACTIVITY,SCHEDULED,2023-06-02,NaN,NaN,NaN,NaN
4,83.0,110310A23CB0071,Projet d'aquisition du local bureau Korhogo,ACTIVITY,SCHEDULED,2024-01-31,NaN,NaN,NaN,NaN
5,84.0,110308A23CB0069,Projet d'aquisition du local bureau SP,ACTIVITY,SCHEDULED,2023-06-02,NaN,NaN,NaN,NaN
6,85.0,110309A23CB0070,Projet d'aquisition du local bureau Yakro,ACTIVITY,SCHEDULED,2024-01-31,NaN,NaN,NaN,NaN
7,86.0,110301A23CB0062,Projet d'aquisition du siège,ACTIVITY,SCHEDULED,2023-12-29,NaN,NaN,NaN,NaN
8,115.0,110835A23CB0035,Activité 7.2.2 : Elaboration du cadre organiqu...,ACTIVITY,SCHEDULED,2023-01-20,NaN,NaN,NaN,NaN
9,116.0,110836A23CB0036,Activité 7.2.3 : Elaboration d'un manuel de pr...,ACTIVITY,SCHEDULED,2023-01-31,NaN,NaN,NaN,NaN


### 1.4 Resources list

In [84]:
emp_task_list = data_resources[["task.taskId","position.employee.employeeId","position.employee.lastName","position.employee.firstName","role","task.name","task.status"]]
rename_col = ["taskId","employeeId","lastName","firstName","role","taskName","status"]
emp_task_list.columns = rename_col
#emp_task_list

KeyError: "None of [Index(['task.taskId', 'position.employee.employeeId',\n       'position.employee.lastName', 'position.employee.firstName', 'role',\n       'task.name', 'task.status'],\n      dtype='object')] are in the [columns]"

In [ ]:
emp_task_list

In [ ]:
# compute total number of tasks per employee
total_tasks = emp_task_list.groupby('employeeId')['taskId'].count()
# compute number of finished tasks per employee
finished_tasks = emp_task_list[emp_task_list['status'] == 'FINISHED'].groupby('employeeId')['taskId'].count()
# compute ratio of finished tasks per employee
ratio_finished = finished_tasks / total_tasks
# merge the three results into a single DataFrame
result = pd.concat([total_tasks, finished_tasks, ratio_finished], axis=1)
result.columns = ['total_tasks', 'finished_tasks', 'ratio_finished']
result.reset_index(inplace=True)

pd.merge(left=result, right=emp_task_list.groupby("employeeId")[["lastName","firstName","role"]].first().reset_index(), how='left')